In [1]:
# HIDDEN CELL
import sys, os

# Importing argopy in dev mode:
on_rtd = os.environ.get('READTHEDOCS', None) == 'True'
if not on_rtd:
    sys.path.insert(0, "/Users/gmaze/git/github/euroargodev/argopy")
    import git
    import argopy
    from argopy.options import OPTIONS
    print("argopy:", argopy.__version__, 
          "\nsrc:", argopy.__file__, 
          "\nbranch:", git.Repo(search_parent_directories=True).active_branch.name, 
          "\noptions:", OPTIONS)
else:
    sys.path.insert(0, os.path.abspath('..'))

import xarray as xr
# xr.set_options(display_style="html");
xr.set_options(display_style="text");

argopy: 0.1.3 
src: /Users/gmaze/git/github/euroargodev/argopy/argopy/__init__.py 
branch: fsspec 
options: {'src': 'erddap', 'local_ftp': '.', 'dataset': 'phy', 'cachedir': '/Users/gmaze/.cache/argopy', 'mode': 'standard'}


In [2]:
import argopy
from argopy import DataFetcher as ArgoDataFetcher

# Performance

## Caching data

- with **argopy** global options:

In [3]:
argopy.set_options(cachedir='mycache_folder')

- in a temporary context:

In [4]:
with argopy.set_options(cachedir='mycache_folder'):
    ds = ArgoDataFetcher(cache=True).profile(6902746, 34).to_xarray()

- when instantiating the data fetcher:

In [5]:
ds = ArgoDataFetcher(cache=True, cachedir='mycache_folder').profile(6902746, 34).to_xarray()

Specifyng a cache directory at the fetcher level will ensure

## Clearing the cache

Cached data have an expiration time of 1 day.

If you want to manuallt clear your cache folder, and/or make sure your data are newly fetched, you can do it at the fetcher level with the ``clear_cache`` method.

Start to fetch data and store them in cache:

In [6]:
fetcher = ArgoDataFetcher(cache=True, cachedir='mycache_folder').profile(6902746, 34)
fetcher.to_xarray();

Fetched data are in the local cache folder:

In [7]:
os.listdir('mycache_folder')

['cache', 'e82db08c95e01f9a522786783c6d377567e6ff26c929eb46f77fe9f6da97445a']

where we see one hash entries the newly fetched data and the cache registry file ``cache``.

We can then fetch something else:

In [8]:
fetcher2 = ArgoDataFetcher(cache=True, cachedir='mycache_folder').profile(1901393, 1)
fetcher2.to_xarray();

All fetched data are now cached in 'mycache_folder':

In [9]:
os.listdir('mycache_folder')

['daae074fd57e5237b9d74f50561b48b77fcfbb830071c625a0655cfa6960b0df',
 'cache',
 'e82db08c95e01f9a522786783c6d377567e6ff26c929eb46f77fe9f6da97445a']

Note the new hash file from the ``fetcher2`` data.

We can safely clear the cache from the first fetcher data:

In [10]:
fetcher.clear_cache()

In [11]:
os.listdir('mycache_folder')

['daae074fd57e5237b9d74f50561b48b77fcfbb830071c625a0655cfa6960b0df', 'cache']

By using the fetcher level clear cache, you make sure that only data fetched with it are removed, while other fetched data (with other fetchers for instance) will stay in place.

If you want to clear the entire cache folder, whatever the fetcher used, do it at the package level with:

In [12]:
argopy.clear_cache()

In [13]:
os.listdir('mycache_folder')

FileNotFoundError: [Errno 2] No such file or directory: 'mycache_folder'